In [21]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import texthero as hero
from nltk.sentiment.vader import SentimentIntensityAnalyzer



In [2]:
def embed_glove(model, s):
    #print(s)
    return_ls = []
    for sentence in s:
        array = []
        for word in str(sentence).split():
            try:
                array.append(model.get_vector(word))
            except:
                pass
        array = np.array(array)
        #print(array.shape)
        return_ls.append(array.mean(axis = 0))
    return return_ls

In [3]:
glove_filename='glove.6B.100d.txt'

word2vec_output_file = glove_filename+'.word2vec'
model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

In [6]:
df = pd.read_csv('TaylorSwift.csv')
df['embedded'] = embed_glove(model, df['Lyric'])
df

,Unnamed: 0,Artist,Title,Album,Year,Date,Lyric,embedded
0,0,Taylor Swift,​cardigan,folklore,2020.0,2020-07-24,vintage tee brand new phone high heels on cobb...,"[-0.04936442, 0.22076005, 0.3989212, -0.381027..."
1,1,Taylor Swift,​exile,folklore,2020.0,2020-07-24,justin vernon i can see you standing honey wit...,"[-0.08032562, 0.27168834, 0.39763886, -0.34551..."
2,2,Taylor Swift,Lover,Lover,2019.0,2019-08-16,we could leave the christmas lights up 'til ja...,"[-0.13587536, 0.2712521, 0.45893988, -0.356685..."
3,3,Taylor Swift,​the 1,folklore,2020.0,2020-07-24,i'm doing good i'm on some new shit been sayin...,"[-0.074463576, 0.28702453, 0.44641495, -0.3564..."
4,4,Taylor Swift,Look What You Made Me Do,reputation,2017.0,2017-08-25,i don't like your little games don't like your...,"[-0.10272202, 0.27085683, 0.55602795, -0.45260..."
...,...,...,...,...,...,...,...,...
474,474,Taylor Swift,Teardrops on my Guitar (Live from Clear Channe...,Live From Clear Channel Stripped 2008,2008.0,2008-06-28,drew looks at me i fake a smile so he won't se...,"[-0.012135044, 0.2152403, 0.45065263, -0.32267..."
475,475,Taylor Swift,Evermore [Forward],NaN,2020.0,2020-12-11,to put it plainly we just couldnt stop writing...,"[-0.11120423, 0.19682164, 0.36371264, -0.23260..."
476,476,Taylor Swift,Welcome Back Grunwald,NaN,NaN,NaN,turn wycd on you're on your grunwald back from...,"[-0.10446095, 0.049317565, 0.46583098, -0.4045..."
477,477,Taylor Swift,Tolerate it (Polskie Tłumaczenie),NaN,2020.0,2020-12-11,zwrotka siedzę i patrzę jak czytasz z głową p...,"[-0.15699837, 0.036558278, 0.2239443, -0.10477..."


In [7]:
df['tsne'] = hero.tsne( df['embedded'])
df = df.dropna()
df

,Unnamed: 0,Artist,Title,Album,Year,Date,Lyric,embedded,tsne
0,0,Taylor Swift,​cardigan,folklore,2020.0,2020-07-24,vintage tee brand new phone high heels on cobb...,"[-0.04936442, 0.22076005, 0.3989212, -0.381027...","[-2.8055171966552734, -2.7598016262054443]"
1,1,Taylor Swift,​exile,folklore,2020.0,2020-07-24,justin vernon i can see you standing honey wit...,"[-0.08032562, 0.27168834, 0.39763886, -0.34551...","[-5.641112804412842, -1.4198912382125854]"
2,2,Taylor Swift,Lover,Lover,2019.0,2019-08-16,we could leave the christmas lights up 'til ja...,"[-0.13587536, 0.2712521, 0.45893988, -0.356685...","[-25.46312713623047, -5.804000377655029]"
3,3,Taylor Swift,​the 1,folklore,2020.0,2020-07-24,i'm doing good i'm on some new shit been sayin...,"[-0.074463576, 0.28702453, 0.44641495, -0.3564...","[-11.805758476257324, 0.7628572583198547]"
4,4,Taylor Swift,Look What You Made Me Do,reputation,2017.0,2017-08-25,i don't like your little games don't like your...,"[-0.10272202, 0.27085683, 0.55602795, -0.45260...","[-16.157672882080078, -24.54958724975586]"
...,...,...,...,...,...,...,...,...,...
470,470,Taylor Swift,Teardrops on My Guitar (Live from SoHo),Live from SoHo - EP,2008.0,2008-01-15,drew looks at me i fake a smile so he won't s...,"[-0.01307675, 0.2213254, 0.45528087, -0.320209...","[-7.291515827178955, -4.465409755706787]"
471,471,Taylor Swift,Picture To Burn (Live from Clear Channel Strip...,Live From Clear Channel Stripped 2008,2008.0,2008-07-28,state the obvious i didn't get my perfect fant...,"[-0.07651639, 0.23102626, 0.4662703, -0.386980...","[7.242705345153809, -10.132895469665527]"
472,472,Taylor Swift,Rare Peer (Brenda Lee Tribute),Woman Walk The Line,2017.0,2017-09-20,i'm curled up in my mother's bed staring inten...,"[-2.4537117e-05, 0.14429724, 0.34610268, -0.22...","[9.790163040161133, 24.44515609741211]"
473,473,Taylor Swift,Should’ve Said No (Live from Clear Channel Str...,Live From Clear Channel Stripped 2008,2008.0,2008-06-28,it's strange to think the songs we used to sin...,"[-0.060324643, 0.32480863, 0.5295578, -0.40079...","[-18.6934871673584, -16.891639709472656]"


In [20]:
hero.scatterplot(
    df, 
    col='tsne', 
    color='Album', 
    title=""
)

In [18]:
#df['pca'] = hero.pca(df['embedded'])
#df['tfidf'] = hero.tfidf(df['Lyric'])
#df['pca'] = hero.tsne(df['tfidf'])
#hero.scatterplot(
#    df, 
#    col='pca', 
#    color='Album', 
#    title="Year"
#)

/Users/max/.pyenv/versions/3.7.11/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/max/.pyenv/versions/3.7.11/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/max/.pyenv/versions/3.7.11/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [ ]:
def setimentanalyzer(df):
    neg='Negative'
    neu='Neutral'
    pos='Positive'
    negative = []
    neutral = []
    positive = []
    dominant_sentiment=[]
    dominant_sentiment_score=[]
    #Initialize the model
    sid = SentimentIntensityAnalyzer()
    #Iterate for each row of lyrics and append the scores
    for i in df.index:
       
        scores = sid.polarity_scores(df['Lyric'].iloc[i])
        negative.append(scores['neg'])
        neutral.append(scores['neu'])
        positive.append(scores['pos'])
        if scores['neg']>scores['pos']:
            dominant_sentiment_score.append(scores['neg'])
            dominant_sentiment.append(neg)
        elif scores['neg']<scores['pos']:
            dominant_sentiment_score.append(scores['pos'])
            dominant_sentiment.append(pos)
        else:
            dominant_sentiment_score.append(scores['neu'])
            dominant_sentiment.append(neu)
    #Create 5 columns to the main data frame  for each score
    df['negative'] = negative
    df['neutral'] = neutral
    df['positive'] = positive
    df['dominant_sentiment']=dominant_sentiment
    df['dominant_sentiment_score']=dominant_sentiment_score
    return df

df = setimentanalyzer(df)

In [ ]:
hero.scatterplot(
    df, 
    col='tsne', 
    color='negative', 
    title=""
)